In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px  #seemed to be a lot more info on using this for choropleth maps than cartopy. For some reason anacondas refuses to install geopandas
import matplotlib.pyplot as plt


# Attempting to create a choropleth map showing population density in UK local authorities to practice mapping stuff

#counties = json.load(open(r"C:\Users\conno\OneDrive\Documents\pythonstuff\population stuff\Counties_and_Unitary_Authorities__December_2016__Boundaries.geojson")) ignore- this is higher level LAs

UK_districts = json.load(open(r"C:\Users\conno\OneDrive\Documents\pythonstuff\population stuff\data\data\2020UKLAs.json")) # had to convert a shapefile to get an up to date UK LAs shapefile



import plotly.io as pio
pio.renderers.default = 'browser'  #opens up the map in a different tab. Without this, I just get a blank box when I try to run the code

#print (UK_districts['features'][1])

df = pd.read_csv (r'C:\Users\conno\OneDrive\Documents\pythonstuff\population stuff\data\data\UKLApopdata.csv')

LA_id_map = {}
for feature in UK_districts["features"]:
    feature["id"] = feature["properties"]["OBJECTID"] # need to make an id feature for some reason, not sure why
    LA_id_map[feature["properties"]["LAD20CD"]] = feature["id"] #LAD20NM is what the authority names are labelled as in the json
    
#that's a point actually, what is the difference between a json and a geojson? could that be causing an issue?

df["Density"] = df["2019 people per sq. km"].apply(lambda x: int(x.split("/")[0].replace(",", "")))#gets rid of commas in the csv data. I don't need the bit about splitting /, but I "borrowed" this code and it complains if I remove it
df["id"] = df["Code"].apply(lambda x: LA_id_map[x]) #code is an identifier for the LAs in the csv
df["DensityScale"] = np.log10(df["Density"]) #putting the densities on log scale for better comparison

df['Code']
fig = px.choropleth(df,locations="id", geojson=UK_districts, color="DensityScale", color_continuous_scale = "blues", title = "Population Density UK Local Authorities" )
fig.update_geos(fitbounds="locations", visible=False)  #reduces the scope of the produced map to just the UK. It won't load otherwise. No apparent difference between True and False
fig.show()

# not sure what the problem is - dodgy json? 